In [26]:
import azureml.core
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.exceptions import ComputeTargetException
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.train.dnn import TensorFlow
from azureml.train.hyperdrive import *
#from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

import os
import shutil
import urllib
import numpy as np
import matplotlib.pyplot as plt

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.16.0


In [27]:
ws = Workspace.get(name="AML")
exp = Experiment(workspace=ws, name="AML")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
print (exp)

Workspace name: AML
Azure region: westus
Subscription id: 4dc660fa-114b-4613-ae16-cb4015161f3d
Resource group: Udacity-AML
Experiment(Name: AML,
Workspace: AML)


In [28]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

try: 
    # check to see if the cluster exists, if it does, assign it to compute_cluster
    compute_cluster = ComputeTarget(ws, 'compute-cluster')
except ComputeTargetException:
    # create provision
    compute_provision = AmlCompute.provisioning_configuration(vm_size = 'Standard_D2_V2', max_nodes=4)
    # create cluster
    compute_cluster = ComputeTarget.create(ws, 'compute-cluster', compute_provision) 
    # wait
    compute_cluster.wait_for_completion()
    print(compute_target.get_status().serialize())

In [30]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import ScriptRunConfig
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling(
{
    '--C' : choice(0.01,0.1,1,10,100),
    '--max_iter' : choice(25,50,75,100,125,150,175,200),
    'solver' : choice('liblinear','sag','lbfgs', 'saga')
})

#sample_space = {
#    'C': choice(0.01, 0.1, 1, 10, 100),
#    'max_iter' : choice(50,75,100,125,150,175,200),
#    
#}
#ps = RandomParameterSampling(sample_space)

# Specify a Policy
policy = MedianStoppingPolicy(evaluation_interval=1,delay_evaluation=5)
#policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
#policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)

# creating a training directory and copying train.py to that directory
if "training" not in os.listdir():
    os.mkdir("./training")
shutil.copy('./train.py','./training')

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./training',
                     entry_script='train.py',
                     compute_target=compute_cluster)  

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    max_total_runs=50,
                                    max_concurrent_runs=5,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)
print(hyperdrive_config)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
#from azureml.widgets import RunDetails

#hd_submit = exp.submit(config=hyperdrive_config)
#RunDetails(hd_submit).show()
#hd_submit.wait_for_completion(show_output=True)

from azureml.widgets import RunDetails

hyperDrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperDrive_run).show()
hyperDrive_run.wait_for_completion()


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
# Get your best run and save the model from that run.

import joblib
best_run = hyperDrive_run.get_best_run_by_primary_metric()
best_run

In [ ]:
# display best run definition and argument
print(best_run)
print('\n')
best_run.get_details()['runDefinition']['arguments']

In [ ]:
# display best run id and accuracy
print('\n')
print('Best Run Id: ', best_run.id)
print('\n')
best_run.get_metrics(name='Accuracy')

In [ ]:
# files of the best run
best_run.get_file_names()

In [ ]:
#model registration of the best run
model = best_run.register_model(model_name='best_run',model_path='outputs/model.joblib')

print(model)

In [ ]:
#Auto ML preparation

from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

dataset_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(path=dataset_path)
print(dataset)

In [ ]:
# Use the clean_data function to clean your data.
from train import clean_data
import pandas as pd

x, y = clean_data(dataset)
predict_col = pd.DataFrame(y)
predict_col.head()
complete_data = pd.concat([x,predict_col],axis=1)
complete_data.head()

In [ ]:
# Split data
from sklearn.model_selection import train_test_split
train_dataset, test_dataset = train_test_split(complete_data,test_size=0.3, random_state=1, shuffle=True)
train_dataset.head()


In [ ]:
#upload to datastore
train_dataset.to_csv('train_bankmarket.csv', index=False)
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train_bankmarket.csv'])

In [ ]:
# Read csv file
data_store = ws.get_default_datastore()
data_store.upload(src_dir='./',target_path='training/')
training_data = Dataset.Tabular.from_delimited_files(path = [(data_store, '/train_bankmarket.csv')])

In [ ]:
# Set parameters for AutoMLConfig
from azureml.train.automl import AutoMLConfig


# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    compute_target=compute_cluster,
    experiment_timeout_minutes=30,
    task='classification', 
    primary_metric='accuracy',
    training_data=training_data,
    label_column_name='y',
    iterations=50,
    max_concurrent_iterations = 5,
    enable_early_stopping = True,
    n_cross_validations=7)
print(automl_config)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.widgets import RunDetails
automl_run = exp.submit(automl_config, show_output = True)
RunDetails(automl_run).show()

In [ ]:
automl_run.wait_for_completion()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl, best_model = automl_run.get_output()
best_automl.register_model(model_name = 'best_automl_model', model_path = '/outputs/')
print(best_automl)

In [ ]:
#Save best automl model
import joblib
joblib.dump(best_model,'outputs/automlmodel.joblib')
best_automl.register_model(model_name = 'best_automl', model_path = '/outputs/')

In [ ]:
#Best accuracy display
best_automl.get_metrics(name='accuracy')

In [ ]:
compute_target.delete()